In [1]:
# TODO:
#  -check X chr qc
#  -generated quantities
#  -speed up

In [2]:
require('rstan')
require('tidyverse')

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.4.2     ✔ purrr   0.2.4
✔ tidyr   0.8.1     ✔ dplyr   0.7.4
✔ readr   1.1.1     ✔ stringr 1.3.0
✔ tibble  1.4.2     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks rstan::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()


### Extract Testosterone SNP of interest

In [8]:

# read in the testosterone files
c1.m <- read_tsv("/scratch/groups/mrivas/users/erflynn/sex_div_gwas/gwas_age_sex/ukb24893_v2_c1.Testosterone.onesex.Testosterone.glm.linear")
c1.f <- read_tsv("/scratch/groups/mrivas/users/erflynn/sex_div_gwas/gwas_age_sex/ukb24893_v2_c1.Testosterone.zerosex.Testosterone.glm.linear")
c1.pre <- read_tsv("/scratch/groups/mrivas/users/erflynn/sex_div_gwas/gwas_age_sex/ukb24893_v2_c1.Testosterone.pre_meno.Testosterone.glm.linear")
c1.post <- read_tsv("/scratch/groups/mrivas/users/erflynn/sex_div_gwas/gwas_age_sex/ukb24893_v2_c1.Testosterone.post_meno.Testosterone.glm.linear")


Parsed with column specification:
cols(
  `#CHROM` = col_integer(),
  POS = col_integer(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  TEST = col_character(),
  OBS_CT = col_integer(),
  BETA = col_double(),
  SE = col_double(),
  T_STAT = col_double(),
  P = col_double()
)
Parsed with column specification:
cols(
  `#CHROM` = col_integer(),
  POS = col_integer(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  TEST = col_character(),
  OBS_CT = col_integer(),
  BETA = col_double(),
  SE = col_double(),
  T_STAT = col_double(),
  P = col_double()
)
Parsed with column specification:
cols(
  `#CHROM` = col_integer(),
  POS = col_integer(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  TEST = col_character(),
  OBS_CT = col_integer(),
  BETA = col_double(),
  SE = col_double(),
  T_STAT = col_double(),
  P = col_double()
)
Parsed with column specification:
cols(
  `#CHROM` = col_integer(),
  POS = col

In [13]:
# filter for the variant
m <- c1.m %>% filter(ID == "rs1553921" & TEST %in% c("ADD", "age") ) %>% mutate(pop="male")
f <- c1.f %>% filter(ID == "rs1553921" & TEST %in% c("ADD", "age")) %>% mutate(pop="female")
pre <- c1.pre %>% filter(ID == "rs1553921" & TEST %in% c("ADD", "age")) %>% mutate(pop="pre_meno")
post <- c1.post %>% filter(ID == "rs1553921" & TEST %in% c("ADD", "age"))%>% mutate(pop="post_meno")

snp_df <- do.call(rbind, list(m, f, pre, post))
snp_df %>% arrange(TEST, pop)

#CHROM,POS,ID,REF,ALT,TEST,OBS_CT,BETA,SE,T_STAT,P,pop
1,204438643,rs1553921,T,G,ADD,122906,0.045803200,0.017091200,2.679930,7.36478e-03,female
1,204438643,rs1553921,T,G,ADD,146149,0.002088240,0.015825500,0.131954,8.95021e-01,male
1,204438643,rs1553921,T,G,ADD,102361,0.070860800,0.012631500,5.609840,2.03034e-08,post_meno
1,204438643,rs1553921,T,G,ADD,47771,-0.069506300,0.014299200,-4.860840,1.17261e-06,pre_meno
1,204438643,rs1553921,T,G,age,122906,0.000529176,0.000356689,1.483580,1.37923e-01,female
1,204438643,rs1553921,T,G,age,146149,0.001360850,0.000323272,4.209610,2.55964e-05,male
1,204438643,rs1553921,T,G,age,102361,0.004951610,0.000557807,8.876920,6.98134e-19,post_meno
1,204438643,rs1553921,T,G,age,47771,0.005661810,0.001427120,3.967310,7.27945e-05,pre_meno


### X Chromosome QC

In [10]:
chrX <- read.delim("../data/chr_qc/chrX_qc_table.txt", sep=" ", header=TRUE)

head(chrX)

chrXY <- read.delim("../data/chr_qc/chrXY_qc_table.txt", sep=" ", header=TRUE)
head(chrXY)

table(chrX$keep==1)
table(chrXY$keep==1)


# XY should be HWE same
# why is there a variant filter for X chromosome and not the others?

SNP,CHR,A1,A2,MAF,F_MISS,LD,keep
Affx-34464730,X,A,G,0.821453,0.000683898,0,0
Affx-34470541,X,T,G,0.998817,0.001566410,1,1
Affx-34480971,X,A,C,0.987194,0.010207300,1,1
Affx-34492707,X,T,C,0.988038,0.002023030,1,1
Affx-34492709,X,G,A,0.224857,0.000788325,0,0
Affx-34494619,X,C,G,0.767836,0.001820320,0,0


SNP,CHR,A1,A2,MAF,F_MISS,LD,keep
Affx-34637689,XY,T,C,0.701461,0.003255680,0,0
Affx-34642957,XY,T,G,0.925713,0.000702326,0,0
Affx-34660664,XY,A,T,0.978064,0.002043500,0,0
Affx-34691882,XY,T,C,0.418119,0.001973880,0,0
Affx-34729416,XY,T,C,0.896423,0.835355000,0,0
Affx-34734836,XY,G,A,0.189222,0.067742700,0,0



FALSE  TRUE 
13045  5812 


FALSE  TRUE 
  829   528 

From plink2 help:
    chrom: Chromosome ID.
    pos: Base-pair coordinate.
    (ID is always present, and positioned here.)
    ref: Reference allele.
    alt1: Alternate allele 1.
    alt: All alternate alleles, comma-separated.
    gcounts: Hom-ref count, total number of ref-altx heterozygous calls, and
             total number of nonmissing calls with no reference allele.  On
             chrX, these are followed by male ref and male alt counts.
    gcount1col: gcounts values in a single comma-separated column.
    hetfreq: Observed and expected heterozygote frequencies.
    sexaf: Female and male ref allele frequencies (chrX only).
    femalep: Female-only p/midp-value (chrX only).
    p: Hardy-Weinberg equilibrium exact test p/midp-value.

In [20]:
hardyX <- read.table("../data/chr_qc/hardy_X.out.hardy.x", sep="", header=FALSE)
colnames(hardyX) <- c("chr", "ID", "ref", "alt", "hom-ref_count", "ref-alt_het", "nmiss", "male_ref_count", "male_alt_count", 
                        "hetfreq","sexaf", "femalep", "p")
head(hardyX)

hardyXY <- read.table("../data/chr_qc/hardy_XY.out.hardy", sep="", header=FALSE)
colnames(hardyXY) <- c("chr", "ID", "ref", "alt", "hom-ref_count", "ref-alt_het", "nmiss", "hetfreq", "p")

head(hardyXY)


V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
X,rs60075487,A,G,7340,72122,185023,36077,186736,2.72688e-01,2.74337e-01,1.64096e-01,0.1619160,5.58886e-04
X,rs2306736,C,T,75908,130843,56870,118635,103318,4.96330e-01,4.97392e-01,5.36109e-01,0.5345050,2.44207e-01
X,Affx-92044070,AT,A,0,20,237148,0,196223,8.43284e-05,8.43249e-05,4.21642e-05,0.0000000,2.17114e-03
X,rs5939320,G,A,26848,114413,122715,70724,151757,4.33422e-01,4.34055e-01,3.18417e-01,0.3178880,6.81506e-01
X,rs146462965,T,C,5,6222,258426,2581,220674,2.35100e-02,2.32706e-02,1.17739e-02,0.0115608,4.90229e-10
X,rs1970797,T,C,198341,60873,4852,192439,30333,2.30522e-01,2.31554e-01,8.66365e-01,0.8638380,1.00389e-03


V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
XY,rs34557243,A,C,1526,34508,450367,0.0709456,0.0742388,1.15550e-163
XY,rs28494123,G,A,136003,230107,117847,0.4754700,0.4992960,1.04200e-241
XY,rs28590175,G,A,4117,37497,446013,0.0768969,0.0893851,0.00000e+00
XY,rs117654552,G,C,1714,35077,450347,0.0720063,0.0759194,3.85634e-215
XY,rs28491545,C,A,136946,233527,116337,0.4797090,0.4991040,6.63162e-162
XY,rs73174453,T,G,141590,240549,104614,0.4941910,0.4971150,4.07944e-05


### GENERATED QUANTITIES